### Notebook used to better understand the data

In [3]:
from pyspark.sql import SparkSession
from dataproc.github_insights_pyspark import GGiGitHubInsights


spark = SparkSession.builder.getOrCreate()
file_path = 'dataproc/testdata/2017-03-01-0.json.gz' 

In [141]:
ggi = GGiGitHubInsights(spark,'')
raw_df= ggi.read_json_gzipped('dataproc/testdata/2017-03-01-0.json.gz') 

In [142]:
raw_df.printSchema()

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- display_login: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- pull_request: struct (nul

#### Cleaning and parsing of data
GitHub events: https://developer.github.com/v3/activity/event_types/#pushevent

In [143]:
from pyspark.sql import functions as F

df = raw_df.where(F.col('type') == 'PushEvent') \
.select(
    F.col('created_at'),  F.col('repo.id').alias('repository_id'), F.col('payload'),
    F.col('payload.distinct_size').alias('dist_commits')
)

df.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- repository_id: long (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- pull_request: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- self: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |-- body: string (nullable = true)
 |    |    |-- commit_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- diff_hunk: string (nullable = true)
 |    |    |-- html_url: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- issue_url: string (nullable = true)
 |    |    |-- line: 

In [144]:
df.where(F.col('payload.distinct_size').isNull()).show()

+----------+-------------+-------+------------+
|created_at|repository_id|payload|dist_commits|
+----------+-------------+-------+------------+
+----------+-------------+-------+------------+



We need repository id, the number of commits

In [147]:
sh = df.select(
    F.col('created_at'),
    F.col('repository_id'),
    F.col('dist_commits'),
    F.to_timestamp(F.col('created_at'), "yyyy-MM-dd'T'HH:mm:ss'Z'").alias('timestamp')
)

sh.show()

+--------------------+-------------+------------+-------------------+
|          created_at|repository_id|dist_commits|          timestamp|
+--------------------+-------------+------------+-------------------+
|2017-03-01T00:00:00Z|     69984295|           1|2017-03-01 00:00:00|
|2017-03-01T00:00:00Z|     82382846|           1|2017-03-01 00:00:00|
|2017-03-01T00:00:00Z|     80294987|           2|2017-03-01 00:00:00|
|2017-03-01T00:00:00Z|     22484898|           2|2017-03-01 00:00:00|
|2017-03-01T00:00:01Z|     77256549|           1|2017-03-01 00:00:01|
|2017-03-01T00:00:01Z|     77859525|           1|2017-03-01 00:00:01|
|2017-03-01T00:00:01Z|     35569859|           0|2017-03-01 00:00:01|
|2017-03-01T00:00:02Z|     78353711|           1|2017-03-01 00:00:02|
|2017-03-01T00:00:02Z|     80152527|           1|2017-03-01 00:00:02|
|2017-03-01T00:00:02Z|     43618452|           9|2017-03-01 00:00:02|
|2017-03-01T00:00:02Z|     15193430|           1|2017-03-01 00:00:02|
|2017-03-01T00:00:03

Add week of year and month. For January it might return 52: https://stackoverflow.com/questions/49904570/weekofyear-returning-seemingly-incorrect-results-for-january-1

In [133]:
final_df = sh.select(
    F.col('repository_id'),
    F.col('dist_commits'),
    F.col('timestamp'),
    F.weekofyear(F.col('timestamp')).alias('weekofyear'),
    F.month(F.col('timestamp')).alias('month'),
)

final_df.show()

+-------------+------------+-------------------+----------+-----+
|repository_id|dist_commits|          timestamp|weekofyear|month|
+-------------+------------+-------------------+----------+-----+
|     77742429|           1|2017-01-01 00:00:00|        52|    1|
|     74242392|           1|2017-01-01 00:00:00|        52|    1|
|     47018576|           1|2017-01-01 00:00:00|        52|    1|
|     77715589|           1|2017-01-01 00:00:01|        52|    1|
|     77742423|           1|2017-01-01 00:00:01|        52|    1|
|     77759709|           2|2017-01-01 00:00:01|        52|    1|
|     73930998|           1|2017-01-01 00:00:02|        52|    1|
|     77181687|           1|2017-01-01 00:00:02|        52|    1|
|     76987651|           1|2017-01-01 00:00:03|        52|    1|
|     77742406|           1|2017-01-01 00:00:03|        52|    1|
|     72613553|           1|2017-01-01 00:00:03|        52|    1|
|     69527012|           1|2017-01-01 00:00:03|        52|    1|
|     7689

In [125]:
raw_df.select(F.col('type')).distinct().show()

+--------------------+
|                type|
+--------------------+
|           PushEvent|
|         GollumEvent|
|        ReleaseEvent|
|  CommitCommentEvent|
|         CreateEvent|
|PullRequestReview...|
|   IssueCommentEvent|
|         DeleteEvent|
|         IssuesEvent|
|           ForkEvent|
|         PublicEvent|
|         MemberEvent|
|          WatchEvent|
|    PullRequestEvent|
+--------------------+



Some ideas for testing

In [134]:
t_df = final_df.sort('timestamp','repository_id')
t_df.take(10)

[Row(repository_id=47018576, dist_commits=1, timestamp=datetime.datetime(2017, 1, 1, 0, 0), weekofyear=52, month=1),
 Row(repository_id=74242392, dist_commits=1, timestamp=datetime.datetime(2017, 1, 1, 0, 0), weekofyear=52, month=1),
 Row(repository_id=77742429, dist_commits=1, timestamp=datetime.datetime(2017, 1, 1, 0, 0), weekofyear=52, month=1),
 Row(repository_id=77715589, dist_commits=1, timestamp=datetime.datetime(2017, 1, 1, 0, 0, 1), weekofyear=52, month=1),
 Row(repository_id=77742423, dist_commits=1, timestamp=datetime.datetime(2017, 1, 1, 0, 0, 1), weekofyear=52, month=1),
 Row(repository_id=77759709, dist_commits=2, timestamp=datetime.datetime(2017, 1, 1, 0, 0, 1), weekofyear=52, month=1),
 Row(repository_id=73930998, dist_commits=1, timestamp=datetime.datetime(2017, 1, 1, 0, 0, 2), weekofyear=52, month=1),
 Row(repository_id=77181687, dist_commits=1, timestamp=datetime.datetime(2017, 1, 1, 0, 0, 2), weekofyear=52, month=1),
 Row(repository_id=15526570, dist_commits=1, time

In [149]:
print(final_df.count())

7898


In [151]:
print(final_df.na.drop().count())

7898
